In [9]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import eurostat
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

In [3]:
avia_par_countries = [
    "be", "bg", "dk", "de", "ee", "ie", "el", "es", "fr", "hr",
    "it", "cy", "lv", "lt", "lu", "hu", "mt", "nl", "at", "pl",
    "pt", "ro", "si", "sk", "fi", "se", "is", "no", "ch", "uk",
    "ba", "me", "mk", "rs", "tr"
]

In [6]:
data = eurostat.get_data_df("avia_par_de")
data.head(10)

,freq,unit,tra_meas,airp_pr\TIME_PERIOD,1993,1994,1995,1996,1997,1998,...,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2024-Q1,2024-Q2,2024-Q3,2024-Q4
0,A,FLIGHT,CAF_PAS,DE_ED00_DE_EDDB,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A,FLIGHT,CAF_PAS,DE_ED00_DE_EDDH,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A,FLIGHT,CAF_PAS,DE_ED00_DE_EDDT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A,FLIGHT,CAF_PAS,DE_EDDB_AE_OMAA,NaN,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A,FLIGHT,CAF_PAS,DE_EDDB_AL_LATI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,A,FLIGHT,CAF_PAS,DE_EDDB_AT_LOWS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,A,FLIGHT,CAF_PAS,DE_EDDB_AT_LOWW,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,A,FLIGHT,CAF_PAS,DE_EDDB_BE_EBBR,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,A,FLIGHT,CAF_PAS,DE_EDDB_BG_LBBG,284.0,601.0,228.0,317.0,295.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,A,FLIGHT,CAF_PAS,DE_EDDB_BG_LBSF,7.0,7.0,14.0,6.0,21.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
print(data)


      freq    unit    tra_meas airp_pr\TIME_PERIOD  1993  1994  1995  1996  \
0        A  FLIGHT     CAF_PAS     DE_ED00_DE_EDDB   NaN   NaN   NaN   NaN   
1        A  FLIGHT     CAF_PAS     DE_ED00_DE_EDDH   NaN   NaN   NaN   NaN   
2        A  FLIGHT     CAF_PAS     DE_ED00_DE_EDDT   NaN   NaN   NaN   NaN   
3        A  FLIGHT     CAF_PAS     DE_EDDB_AE_OMAA   NaN   NaN   2.0   NaN   
4        A  FLIGHT     CAF_PAS     DE_EDDB_AL_LATI   NaN   NaN   NaN   NaN   
...    ...     ...         ...                 ...   ...   ...   ...   ...   
75839    Q    SEAT  ST_PAS_DEP     DE_ETNL_ES_LEMD   NaN   NaN   NaN   NaN   
75840    Q    SEAT  ST_PAS_DEP     DE_ETNL_ES_LEPA   NaN   NaN   NaN   NaN   
75841    Q    SEAT  ST_PAS_DEP     DE_ETNL_IT_LIMC   NaN   NaN   NaN   NaN   
75842    Q    SEAT  ST_PAS_DEP     DE_ETNL_IT_LIRF   NaN   NaN   NaN   NaN   
75843    Q    SEAT  ST_PAS_DEP     DE_ETNL_TR_LTAI   NaN   NaN   NaN   NaN   

       1997  1998  ...  2024-07  2024-08  2024-09  2024-10  202

In [8]:
# filter rows where unit is 'PAS' and freq is 'Q'
data[(data['unit'] == 'PAS') & (data['freq'] == 'Q')].head(10)

,freq,unit,tra_meas,airp_pr\TIME_PERIOD,1993,1994,1995,1996,1997,1998,...,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2024-Q1,2024-Q2,2024-Q3,2024-Q4
57676,Q,PAS,PAS_BRD,DE_ED00_DE_EDDB,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57677,Q,PAS,PAS_BRD,DE_ED00_DE_EDDH,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57678,Q,PAS,PAS_BRD,DE_ED00_DE_EDDT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57679,Q,PAS,PAS_BRD,DE_EDDB_AL_LATI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,19470.0,34863.0,31126.0,30413.0
57680,Q,PAS,PAS_BRD,DE_EDDB_AT_LOWS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57681,Q,PAS,PAS_BRD,DE_EDDB_AT_LOWW,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,119210.0,218692.0,196399.0,160123.0
57682,Q,PAS,PAS_BRD,DE_EDDB_BE_EBBR,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,92747.0,104168.0,103611.0,99674.0
57683,Q,PAS,PAS_BRD,DE_EDDB_BG_LBBG,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57684,Q,PAS,PAS_BRD,DE_EDDB_BG_LBSF,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,27032.0,36420.0,35080.0,30458.0
57685,Q,PAS,PAS_BRD,DE_EDDB_BG_LBWN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
